In [ ]:
import numpy as np
import dreamcoder as dc
from dreamcoder.domains.quantum_algorithms.primitives import *
from dreamcoder.domains.quantum_algorithms.tasks import *
from qiskit.transpiler.synthesis import solovay_kitaev
skd = solovay_kitaev.SolovayKitaevDecomposition()

import time
from tqdm import trange
import random
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [ ]:
# This branch of qiskit is required:  pip install git+https://github.com/LNoorl/qiskit-terra.git@feature/sk-pass

In [ ]:
basis_gates=['h',"cx",'t',"tdg"]

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.t(0)
    QT.circuit.t(0)
qk.circuit.equivalence_library.StandardEquivalenceLibrary.add_equivalence(qk.circuit.library.SGate(),QT.circuit)

with QiskitTester(1) as QT:
    QT.circuit.tdg(0)
    QT.circuit.tdg(0)
qk.circuit.equivalence_library.StandardEquivalenceLibrary.add_equivalence(qk.circuit.library.SdgGate(),QT.circuit)

class ParametricSubstitution(qk.transpiler.TransformationPass):
    def run(self, dag):
        # iterate over all operations
        for node in dag.op_nodes():
            print(node.op.name, node.op.params)
            # if we hit a RYY or RZZ gate replace it
            
            if node.op.name in ["cp"]:
                replacement = QuantumCircuit(2)
                replacement.p(node.op.params[0]/2,0)
                replacement.cx(0,1)
                replacement.p(-node.op.params[0]/2,1)
                replacement.cx(0,1)
                replacement.p(node.op.params[0]/2,1)

                # replace the node with our new decomposition
                dag.substitute_node_with_dag(node, qk.converters.circuit_to_dag(replacement))
                                
            
            if node.op.name in ["p"] and node.op.params[0]==np.pi/2:

                # calculate the replacement
                replacement = QuantumCircuit(1)
                replacement.s([0])

                # replace the node with our new decomposition
                dag.substitute_node_with_dag(node, qk.converters.circuit_to_dag(replacement))
                
            elif node.op.name in ["p"] and node.op.params[0]==3*np.pi/2:
    
                # calculate the replacement
                replacement = QuantumCircuit(1)
                replacement.tdg([0])
                replacement.tdg([0])

                # replace the node with our new decomposition
                dag.substitute_node_with_dag(node, qk.converters.circuit_to_dag(replacement))
                               
            elif node.op.name in ["p"] and node.op.params[0]==5*np.pi/2:
        
                # calculate the replacement
                replacement = QuantumCircuit(1)
                replacement.t([0])
                replacement.t([0])

                # replace the node with our new decomposition
                dag.substitute_node_with_dag(node, qk.converters.circuit_to_dag(replacement))
                               
                               
        return dag
pm = qk.transpiler.PassManager()
pm.append(ParametricSubstitution())

### Some fixed-size circuits

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.x(QT.q(0))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.y(QT.q(0))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.z(QT.q(0))
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.t(QT.q(0))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(QT.circuit)
QT.circuit.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(3*np.pi/4),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.CPhaseGate(3*np.pi/2),(0,1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.s(0)
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(np.pi/2),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.tdg(0)
    QT.circuit.tdg(0)
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(3*np.pi/2),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(5*np.pi/2),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.sx(QT.q(0))
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.sxdg(QT.q(1))
QT.circuit = pm.run( QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.cy(QT.q(0),QT.q(1))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.cz(QT.q(0),QT.q(1))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.swap(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.iswap(QT.q(0),QT.q(1))
QT.circuit = pm.run( QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# This is CS, but can we transpile it?
with QiskitTester(2) as QT:
    QT.circuit.cp(np.pi/2,QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.SGate().control(1), (0,1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# (just as a test)
# we should be able to transpile this one
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,0)
    QT.circuit.p(np.pi/8,0)
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.ccx(QT.q(0),QT.q(1),QT.q(2))
QT.circuit.draw(),qk.transpile(QT.circuit, backend, basis_gates).draw()
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.dcx(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.ch(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.cswap(QT.q(0),QT.q(1),QT.q(2))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.Permutation(3,[2,0,1]), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit = pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.AND(2), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit = pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.OR(2), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit = pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.InnerProduct(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3)))
QT.circuit = pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(3*np.pi/2),(0,))
QT.circuit = pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# Bell
with QiskitTester(2) as QT:
    QT.circuit.h(QT.q(0))
    QT.circuit.cx(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# GHZ
with QiskitTester(3) as QT:
    QT.circuit.h(QT.q(0))
    QT.circuit.cx(QT.q(0),QT.q(1))
    QT.circuit.cx(QT.q(0),QT.q(2))
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [7]:
# W this is too long
with QiskitTester(3) as QT:
    QT.circuit.ry(2*np.arccos(1/np.sqrt(3)),QT.q(0))
    QT.circuit.h(QT.q(1))
    QT.circuit.cnot(QT.q(1),QT.q(2))
    QT.circuit.cnot(QT.q(0),QT.q(1))
    QT.circuit.x(QT.q(0))
transpiled=qk.transpile(QT.circuit, backend)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw();

In [ ]:
# In general: GHZ and W state preparation algorithms https://arxiv.org/pdf/1807.05572.pdf
# 
# W states: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_qiskit_2018/w_state/W%20State%201%20-%20Multi-Qubit%20Systems.ipynb

In [ ]:
# Cluster states https://www.cs.umd.edu/class/spring2018/cmsc457/report/Cluster_State_Quantum_Computing.pdf

In [ ]:
with QiskitTester(6) as QT:
    QT.circuit.append(qk.circuit.library.CDKMRippleCarryAdder(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(7) as QT:
    QT.circuit.append(qk.circuit.library.VBERippleCarryAdder(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5),QT.q(6)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [11]:
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.WeightedAdder(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

ccx []
cx []
x []
ccx []
ccx []
x []
ccx []
cx []
x []
ccx []
ccx []
x []


(     ┌────────┐
 q_0: ┤4       ├
      │        │
 q_1: ┤3       ├
      │        │
 q_2: ┤2 adder ├
      │        │
 q_3: ┤1       ├
      │        │
 q_4: ┤0       ├
      └────────┘,
      ┌───┐               ┌───┐     ┌───┐               ┌───┐     
 q_0: ┤ X ├────────────■──┤ X ├─────┤ X ├────────────■──┤ X ├─────
      └─┬─┘          ┌─┴─┐└─┬─┘     └─┬─┘          ┌─┴─┐└─┬─┘     
 q_1: ──┼────────────┤ X ├──┼─────────┼────────────┤ X ├──┼───────
        │  ┌───┐┌───┐└─┬─┘  │  ┌───┐  │  ┌───┐┌───┐└─┬─┘  │  ┌───┐
 q_2: ──■──┤ X ├┤ X ├──┼────■──┤ X ├──■──┤ X ├┤ X ├──┼────■──┤ X ├
        │  └─┬─┘└───┘  │    │  └───┘  │  └─┬─┘└───┘  │    │  └───┘
 q_3: ──┼────┼─────────┼────┼─────────■────■─────────■────■───────
        │    │         │    │                                     
 q_4: ──■────■─────────■────■─────────────────────────────────────
                                                                  ,
      ┌───┐                                        ┌───┐                  

In [13]:
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.IntegerComparator(2, value=3), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

cx []
ccx []
cx []


(             
 q_0: ────────
      ┌──────┐
 q_1: ┤3     ├
      │      │
 q_2: ┤2     ├
      │  cmp │
 q_3: ┤1     ├
      │      │
 q_4: ┤0     ├
      └──────┘,
                     
 q_0: ───────────────
      ┌───┐     ┌───┐
 q_1: ┤ X ├──■──┤ X ├
      └─┬─┘┌─┴─┐└─┬─┘
 q_2: ──┼──┤ X ├──┼──
        │  └─┬─┘  │  
 q_3: ──┼────■────┼──
        │         │  
 q_4: ──■─────────■──
                     ,
                     
 q_0: ───────────────
      ┌───┐     ┌───┐
 q_1: ┤ X ├──■──┤ X ├
      └─┬─┘┌─┴─┐└─┬─┘
 q_2: ──┼──┤ X ├──┼──
        │  └─┬─┘  │  
 q_3: ──┼────■────┼──
        │         │  
 q_4: ──■─────────■──
                     )

In [20]:
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.ExactReciprocal(2, scaling=1.5), 
                      (QT.q(0),QT.q(1),QT.q(2)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

cx []
cx []
cx []
cx []


(             
 q_0: ────────
              
 q_1: ────────
      ┌──────┐
 q_2: ┤2     ├
      │      │
 q_3: ┤1 1/x ├
      │      │
 q_4: ┤0     ├
      └──────┘,
                          
 q_0: ────────────────────
                          
 q_1: ────────────────────
      ┌───┐┌───┐┌───┐┌───┐
 q_2: ┤ X ├┤ X ├┤ X ├┤ X ├
      └─┬─┘└─┬─┘└─┬─┘└─┬─┘
 q_3: ──┼────■────┼────■──
        │         │       
 q_4: ──■─────────■───────
                          ,
                          
 q_0: ────────────────────
                          
 q_1: ────────────────────
      ┌───┐┌───┐┌───┐┌───┐
 q_2: ┤ X ├┤ X ├┤ X ├┤ X ├
      └─┬─┘└─┬─┘└─┬─┘└─┬─┘
 q_3: ──┼────■────┼────■──
        │         │       
 q_4: ──■─────────■───────
                          )

In [17]:
# very long!
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.QuadraticForm(2, linear=[1,-2], 
                                                       quadratic=[[1,2],[2,0.5]]), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw();

h []
h []
cp [3.141592653589793]
cp [-2.356194490192345]
cp [6.283185307179586]
cp [-4.71238898038469]
mcphase [6.283185307179586]
mcphase [12.566370614359172]
h []
cp [-1.5707963267948966]
h []


In [ ]:
# we should be able to transpile this one
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,0)
    QT.circuit.p(np.pi/8,0)
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.IQP([[6, 5, 3], [5, 4, 5], [3, 5, 1]]), 
                      (QT.q(0),QT.q(1),QT.q(2)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

### Invalid

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.csx(QT.q(0),QT.q(1))
QT.result
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,QT.q(0))
QT.circuit.draw()
QT.result
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.C3XGate(), (0,1,2,3))
QT.circuit.draw()
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.csx(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.ecr(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:

# no idea of how this works
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.Diagonal([1,1]), 
                      (QT.q(0),))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
## WE NEED CP(pi/8) FOR QFT!
# It works but it is huge!
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.QFT(3), 
                      (QT.q(0),QT.q(1),QT.q(2)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
## WE NEED CP(pi/8) FOR QFT!
# It works but it is huge!
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.QFT(4), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# Huge
# U(π/2,0,π,0)  not found
with QiskitTester(9) as QT:
    QT.circuit.append(qk.circuit.library.HRSCumulativeMultiplier(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5),QT.q(6),QT.q(7),QT.q(8)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
#!Huge
with QiskitTester(8) as QT:
    QT.circuit.append(qk.circuit.library.RGQFTMultiplier(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5),QT.q(6),QT.q(7)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
#incredibly huge
from qiskit import algorithms
circuit = qk.algorithms.Shor().construct_circuit(7)
transpiled=qk.transpile(circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# Other algorithms - but should be evaluated with reasonable arguments!
# https://qiskit.org/documentation/apidoc/algorithms.html